In [ ]:
{-# LANGUAGE ScopedTypeVariables, ExistentialQuantification #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE RankNTypes, BangPatterns #-}
{-# LANGUAGE ForeignFunctionInterface #-}
{-# LANGUAGE OverloadedLists #-}
import Control.Applicative
import Control.Monad
import Control.Monad.Primitive

import Data.List
import qualified Data.Vector.Generic as G
import           Data.Vector.Generic ((!))
import qualified Data.Vector.Unboxed as U

import Numeric.SpecFunctions
import Numeric.SpecFunctions.Extra (logGammaCorrection)
import Numeric.MathFunctions.Constants 
import Numeric.MathFunctions.Comparison
import Numeric.Polynomial
import Numeric.Series
import Numeric.Polynomial.Chebyshev

import IHaskell.Display
import Graphics.Rendering.Chart.Backend.Cairo
import Graphics.Rendering.Chart.Easy

import Debug.Trace
import Text.Printf

:l NB/Plot

In [ ]:
-- Chebyshev polynomials of low order
cheb :: [Rational -> Rational]
cheb =
  [ \_ -> 1
  , \x -> x
  , \x -> 2*x^2 - 1
  , \x -> 4*x^3 - 3*x
  , \x -> 8*x^4 - 8*x^2 + 1
  ]
  
evalCheb :: [Double] -> Double -> Double
evalCheb as x
  = realToFrac
  $ sum
  $ zipWith (*) (map realToFrac as)
  $ map ($ realToFrac x) cheb

In [ ]:
coefs1 = 
 [ 25.78759508324379
 , 2.4353125270813036
 , 11.581499359315602
 , -20.560154201155836
-- ,  2.0394268771113357
 ] :: [Double]
coefs2 = 
 [ 9.169334849013147,
  -50.96992537799096,
   399.8402984262722,
  13.328005219369693
 ] :: [Double]
coefs3 =
 [ 0.0
 ,  0.0
 , 0.0
 , 0.27016867846896275
 , 0.507730912675075 
 ] :: [Double]

coefs = coefs3
toRenderable $
  plotFunctions [\x -> chebyshev x (U.fromList coefs)] (-1,1)
--toRenderable $
--  plotFunctions 
--    [\x -> logBase 10 $ abs $ relativeError (chebyshev x (U.fromList coefs)) (evalCheb coefs x)
--    , const $ logBase 10 $ m_epsilon / 2
--    ]
--    (-1,1)
toRenderable $
 let cc = U.fromList coefs
     nU  = maximum $ map (flip chebyshev cc) $ linspace (-1,1) 100 
     nL  = minimum $ map (flip chebyshev cc) $ linspace (-1,1) 100 
 in plotFunctions 
    [\x -> (chebyshev x cc - evalCheb coefs x) / m_epsilon / nU
    ]
    (-1,1)

In [ ]:
toRenderable $
  plotFunctions
    [ \x -> chebyshev x (U.fromList [1,0,0,0,0])
    , \x -> chebyshev x (U.fromList [0,1,0,0,0])
    , \x -> chebyshev x (U.fromList [0,0,1,0,0])
    , \x -> chebyshev x (U.fromList [0,0,0,1,0])
    , \x -> chebyshev x (U.fromList [0,0,0,0,1])
    ] (-1,1)

In [ ]:
:t plotFunctions

In [ ]:
:t linspace

In [ ]:
 linspace (-1,1) 100 
